In [1]:
# Import the libraries we will use:

import imageio
import numpy as np
import pandas as pd
from PIL import Image
from skimage import data, io
import os

## Step 1
In this step we will convert all coloured monkey photos into black & white ones of a fixed size.
This step can be omitted if the photos are already there.

In [8]:
# Create the Black-White monkey photos from the coloured ones and standardize the shape.
# ======================================================================================

def create_bw_photo(infile, outfile):
    """
    """
    img = Image.open(infile)

    img = img.convert("L").resize((350,350))
    bw_image = np.array(img.getdata(), dtype = np.uint8).reshape(350,350)

    # Sanity check
    if img.size != bw_image.shape:
        print(f"ISSUE when sizing {in_dir + file}")

    io.imsave(outfile, bw_image)
    

# Directories for datasets:
photo_root = "./monkeys"

source_training_photos = photo_root + "/source_dataset/training"
dest_training_photos = photo_root + "/derived_dataset/training"

source_validation_photos = photo_root + "/source_dataset/validation"
dest_validation_photos = photo_root + "/derived_dataset/validation"

# Restrict our analysis to following monkey categories:
monkey_groups = ["/n0", "/n1", "/n2"]

for group in monkey_groups:
    # Process "training" dataset:
    # Create B&W output directory
    os.makedirs(dest_training_photos + group, exist_ok=True)
        
    # What are the colour photo files
    group_files = os.listdir(source_training_photos + group)
    print(f"DEBUG: Processing Group (Training): {group}, with {len(group_files)} files")

    in_dir = source_training_photos + group + "/"
    out_dir = dest_training_photos + group + "/"

    for file in group_files:
        create_bw_photo(in_dir + file, out_dir + file)


    # Process "validation" dataset:
    # Create B&W output directory
    os.makedirs(dest_validation_photos + group, exist_ok=True)

    # What are the colour photo files        
    group_files = os.listdir(source_validation_photos + group)
    print(f"DEBUG: Processing Group (Validation): {group}, with {len(group_files)} files")

    in_dir = source_validation_photos + group + "/"
    out_dir = dest_validation_photos + group + "/"

    for file in group_files:
        create_bw_photo(in_dir + file, out_dir + file)

DEBUG: Processing Group (Training): /n0, with 105 files
DEBUG: Processing Group (Validation): /n0, with 26 files
DEBUG: Processing Group (Training): /n1, with 111 files
DEBUG: Processing Group (Validation): /n1, with 28 files
DEBUG: Processing Group (Training): /n2, with 110 files
DEBUG: Processing Group (Validation): /n2, with 27 files
